Sebuah perusahaan sedang mengimplementasikan sebuah metode penjualan atau periklanan baru pada halaman website. Dengan menggunakan A/B Testing ini, perusahaan ingin melihat apakah strategi metode penjualan atau periklanan baru ini memberikan dampak yang signifikan pada produknya dibandingkan dengan strategi penjualan atau periklanan sebelumnya.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats
from random import randint
from scipy import stats
import io

### **Control Group**

In [ ]:
from google.colab import files
  
  
uploaded = files.upload()

Saving control_group_ab.csv to control_group_ab.csv


In [ ]:
control = pd.read_csv(io.BytesIO(uploaded['control_group_ab.csv']))
control.head()

,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches
0,Control Campaign,1.08.2019,2280,82702,56930,7016,2290
1,Control Campaign,2.08.2019,1757,121040,102513,8110,2033
2,Control Campaign,3.08.2019,2343,131711,110862,6508,1737
3,Control Campaign,4.08.2019,1940,72878,61235,3065,1042
4,Control Campaign,6.08.2019,3083,109076,87998,4028,1709


### **Experiment/Treatment Group**

In [ ]:
from google.colab import files
  
  
uploaded = files.upload()

Saving test_group_ab.csv to test_group_ab.csv


In [ ]:
experiment = pd.read_csv(io.BytesIO(uploaded['test_group_ab.csv']))
experiment.head()

,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches
0,Test Campaign,1.08.2019,3008,39550,35820,3038,1946
1,Test Campaign,2.08.2019,2542,100719,91236,4657,2359
2,Test Campaign,3.08.2019,2365,70263,45198,7885,2572
3,Test Campaign,4.08.2019,2710,78451,25937,4216,2216
4,Test Campaign,6.08.2019,2458,42684,31489,7488,1854


Date = Tanggal/Waktu Pengujian
<br> Spend = Jumlah total uang yang diterima perusahaan terkait penjualan produk
<br> '# of Impressions' = Jumlah user yang melihat iklan
<br> Reach = Jumlah unique id yang melihat iklan
<br> '# of Website Clicks' = Jumlah user yang mengklik iklan
<br> '# of Searches' = Jumlah user yang melakukan pencarian melalui mesin pencarian terkait produk

In [ ]:
control.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Campaign Name        29 non-null     object
 1   Date                 29 non-null     object
 2   Spend [USD]          29 non-null     int64 
 3   # of Impressions     29 non-null     int64 
 4   Reach                29 non-null     int64 
 5   # of Website Clicks  29 non-null     int64 
 6   # of Searches        29 non-null     int64 
dtypes: int64(5), object(2)
memory usage: 1.7+ KB


In [ ]:
experiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Campaign Name        29 non-null     object
 1   Date                 29 non-null     object
 2   Spend [USD]          29 non-null     int64 
 3   # of Impressions     29 non-null     int64 
 4   Reach                29 non-null     int64 
 5   # of Website Clicks  29 non-null     int64 
 6   # of Searches        29 non-null     int64 
dtypes: int64(5), object(2)
memory usage: 1.7+ KB


### **General Method**

In [ ]:
def var_gen(x,y):
    varx = np.var(x,ddof=1)
    var_mean_x = varx/y
    return var_mean_x

### **Delta Method**

In [ ]:
def var_ratio(x,y): #x/y
    mean_x = np.mean(x)
    mean_y = np.mean(y)
    var_x = np.var(x,ddof=1)
    var_y = np.var(y,ddof=1)
    cov_xy = np.cov(x,y,ddof=1)[0][1]
    result = (var_x/mean_x**2 + var_y/mean_y**2 - (2*cov_xy/(mean_x*mean_y)))*(mean_x*mean_x)/(mean_y*mean_y*len(x))
    return result

### **Z-Test**

In [ ]:
def ttest(mean_control,mean_experiment,var_control,var_experiment):
    diff = mean_experiment - mean_control
    var = var_control+var_experiment
    z = diff/np.sqrt(var)
    p_val = scipy.stats.norm.sf(abs(z))

    result = {'mean_control':mean_control,
             'mean_experiment':mean_experiment,
             'var_control':var_control,
             'var_experiment':var_experiment,
             'difference':diff,
             'z_value':z,
             'p-value':p_val}
    return pd.DataFrame(result,index=[0])

### **Pendapatan**

Hipotesis:
<br>
<br> H0 : μ(eksperimen) - μ(control) <= 0
<br> H1 : μ(eksperimen) - μ(control) > 0

<br> Gunakan taraf nyata (α) = 0.05 (5%) dengan menggunakan z-test dikarenakan jumlah sampel yang digunakan berjumlah lebih dari 30

In [ ]:
var_control = var_gen(control['Spend [USD]'],len(control.index))
var_experiment = var_gen(experiment['Spend [USD]'],len(experiment.index))
mean_control = np.mean(control['Spend [USD]'])
mean_experiment = np.mean(experiment['Spend [USD]'])


ttest(mean_control,mean_experiment,var_control,var_experiment)

,mean_control,mean_experiment,var_control,var_experiment,difference,z_value,p-value
0,2304.068966,2572.241379,4557.157466,4252.065653,268.172414,2.857231,0.002137


Dengan α = 0.05 (5%) maka didapatkan z(1-α) = 1.645 yang berarti z(1-α) < z(value).

Sehingga tolak H0 dan terima H1. Keputusan ini juga diperkuat dengan nilai p-value yang bernilai < 0.1 dan juga p-value < α yang berarti adanya bukti kuat untuk menolak H0. Jadi dapat disimpulkan bahwa metode periklanan baru yang digunakan pada halaman website mampu meningkatkan pendapatan dari perusahaan.


### **Clickthrough rate atau CTR**

Hipotesis:
<br>
<br> H0 : μ(eksperimen) - μ(control) <= 0
<br> H1 : μ(eksperimen) - μ(control) > 0

<br> Gunakan taraf nyata (α) = 0.05 (5%) dengan menggunakan z-test dikarenakan jumlah sampel yang digunakan berjumlah lebih dari 30

In [ ]:
var_control = var_ratio(control['# of Website Clicks'],control['# of Impressions'])
var_experiment = var_ratio(experiment['# of Website Clicks'],experiment['# of Impressions'])
mean_control = control['# of Website Clicks'].sum()/control['# of Impressions'].sum()
mean_experiment = experiment['# of Website Clicks'].sum()/experiment['# of Impressions'].sum()


ttest(mean_control,mean_experiment,var_control,var_experiment)

,mean_control,mean_experiment,var_control,var_experiment,difference,z_value,p-value
0,0.048565,0.082471,0.000013,0.000071,0.033906,3.68483,0.000114


Dengan α = 0.05 (5%) maka didapatkan z(1-α) = 1.645 yang berarti z(1-α) < z(value).

Sehingga tolak H0 dan terima H1. Keputusan ini juga diperkuat dengan nilai p-value yang bernilai < 0.1 dan juga p-value < α yang berarti adanya bukti kuat untuk menolak H0. Jadi dapat disimpulkan bahwa metode periklanan baru yang digunakan pada halaman website mampu meningkatkan perilaku pengunjung website untuk mengklik iklan tersebut

### **Pencarian**

Hipotesis:
<br>
<br> H0 : μ(eksperimen) - μ(control) <= 0
<br> H1 : μ(eksperimen) - μ(control) > 0

<br> Gunakan taraf nyata (α) = 0.05 (5%) dengan menggunakan z-test dikarenakan jumlah sampel yang digunakan berjumlah lebih dari 30

In [ ]:
var_control = var_gen(control['# of Searches'],len(control.index))
var_experiment = var_gen(experiment['# of Searches'],len(experiment.index))
mean_control = np.mean(control['# of Searches'])
mean_experiment = np.mean(experiment['# of Searches'])


ttest(mean_control,mean_experiment,var_control,var_experiment)

,mean_control,mean_experiment,var_control,var_experiment,difference,z_value,p-value
0,2221.310345,2429.758621,25865.889417,5272.378461,208.448276,1.181275,0.118747


Dengan α = 0.05 (5%) maka didapatkan z(1-α) = 1.645 yang berarti z(1-α) > z(value).

Sehingga tolak H1 dan terima H0. Keputusan ini juga diperkuat dengan nilai p-value yang bernilai > 0.1 dan juga p-value > α yang berarti tidak adanya bukti kuat untuk menolak H0. Jadi dapat disimpulkan bahwa metode periklanan baru yang digunakan pada halaman website belum mampu meningkatkan hasil pencarian mengenai produk melalui mesin pencarian. Namun, jika dilihat melalui jumlah hasil pencarian per harinya (mean_control dan mean_experiment) maka terjadi peningkatan jumlah pencarian dalam mesin pencarian tetapi kemungkinan besarnya peningkatan hasil pencarian tidak terlalu besar sehingga belum mampu membuktikan secara umum bahwa terjadi peningkatan pencarian dalam mesin pencarian mengenai produk secara signifikan. Sehingga masih diperlukan cara yang lebih menarik untuk membuat produk perusahaan banyak dicari oleh pengguna.

### **Simpulan**

Metode penjualan atau periklanan baru yang sedang diuji coba oleh perusahaan pada halaman website mampu meningkatkan pendapatan perusahaan dan ketertarikan user untuk mengklik atau melihat lebih lanjut mengenai iklan tersebut. Namun, di sisi lain metode baru ini belum mampu meningkatkan jumlah pencarian mengenai produk tersebut pada mesin pencarian secara signifikan. Oleh sebab itu masih diperlukan adanya evaluasi mengenai metode penjualan atau periklanan ini agar dapat meningkatkan jumlah pencarian dalam mesin pencarian.